# Exective summary of Work Package 2

## Objectives

In this WP, you will work on a given training dataset. Your goal is to develop a fault detection model using the classification algorithms learnt in the class, in order to achieve best F1 score.

## Tasks

- Task 1: Develop a fault detection model using the unsupervised learning algorithms learnt in the class, in order to achieve best F1 score.
- Task 2: With the help of the supporting script, develop a cross-validation scheme to test the performance of the developed classification algorithms.
- Task 3: Develop a fault detection model using the classification algorithms learnt in the class, in order to achieve best F1 score.

## Delierables

- A Jupyter notebook reporting the process and results of the above tasks


# Before starting, please:
- Fetch the most up-to-date version of the github repository.
- Create a new branch with your name, based on the "main" branch and switch to your own branch.
- Copy this notebook to the work space of your group, and rename it to TD_WP_2_Your name.ipynb
- After finishing this task, push your changes to the github repository of your group.

# Task 1: Unsupervised learning approaches

## Implement the statistical testing approach for fault detection

In this exercise, we interpret the statistical testing approach for fault detection. The basic idea of statistical testing approach is that we fit a multi-dimensitional distribution to the observation data under normal working condition. Then, when a new data point arrives, we design a hypothesis test to see whether the new data point is consistent with the distribution. If the new data point is consistent with the distribution, we can conclude that the fault is not due to the faulty component.

The benefit of this approach is that, to design the detection algrothim, we do not need failed data. Also, the computational time is short as all we need is just to compute the pdf and compare it to a threshold.

In this exercise, you need to:
- Fit a multi-dimensitional distribution to the training dataset (all normal samples).
- Design a fault detection algorithm based on the fitted distribution to detect faulty components.

The following block defines a few functions that you can use.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import multivariate_normal


def estimateGaussian(X):
    '''Given X, this function estimates the parameter of a multivariate Gaussian distribution.'''
    mu = np.mean(X, axis=0)
    sigma2 = np.var(X, axis=0)
    return mu, sigma2


def classify(X, distribution, log_epsilon=-50):
    '''Given X, this function classifies each sample in X based on the multivariate Gaussian distribution. 
       The decision rule is: if the log pdf is less than log_epsilon, we predict 1, as the sample is unlikely to be from the distribution, which represents normal operation.
    '''
    p = distribution.logpdf(X)
    predictions = (p < log_epsilon).astype(int)
    
    return predictions

Let us use the dataset `20240105_164214` as training dataset, as all the samples in this dataset are normal operation. We will use the dataset `20240325_155003` as testing dataset. Let us try to predict the state of motor 1. For this, we first extract the position, temperature and voltage of motor 1 as features (you can change the features if you want). 

In [3]:
import sys
sys.path.append('../../supporting_scripts/WP_1')

from utility import read_all_csvs_one_test
import pandas as pd
from sklearn.metrics import f1_score

# Specify path to the dictionary.
base_dictionary = '../../dataset/training_data/'
dictionary_name = '20240105_164214'
path = base_dictionary + dictionary_name

# Read the data.
df_data = read_all_csvs_one_test(path, dictionary_name)

# Get the features
X_train = df_data[['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage']]

# We do the same to get the test dataset.
dictionary_name = '20240325_155003'
path = base_dictionary + dictionary_name

# Read the data.
df_data_test = read_all_csvs_one_test(path, dictionary_name)

# Get the features
X_test = df_data[['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage']]
y_test = df_data['data_motor_1_label']

Please design your algorithm below:

In [5]:
### Preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

features_to_keep = ["data_motor_2_temperature", "data_motor_3_position", "data_motor_1_voltage", "data_motor_1_temperature", "data_motor_1_position"]

X_train = df_data.loc[:, features_to_keep]
X_test = df_data_test.loc[:, features_to_keep]
y_test = df_data_test['data_motor_1_label']

### Moving average
X_train_smooth = X_train.copy()
X_test_smooth = X_test.copy()
window_size = 10
for col in X_train.columns:
    X_train_smooth[col] = X_train[col].rolling(window=window_size).mean()
    X_test_smooth[col] = X_test[col].rolling(window=window_size).mean()
X_train_smooth = X_train_smooth.loc[window_size:, :]
X_test_smooth = X_test_smooth.loc[window_size:, :]
y_test = y_test.loc[window_size:]

### Normaliser
scaler = RobustScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train_smooth))
X_test = pd.DataFrame(scaler.transform(X_test_smooth))

# First, we need to fit a MVN distribution to the normal samples.
mu, sigma2 = estimateGaussian(X_train)


# Construct a multivariate Gaussian distribution to represent normal operation.
distribution = multivariate_normal(mean=mu, cov=np.diag(sigma2), allow_singular=True)

# Now, let's try to predict the labels of the test set X_test.
max_score = 0
k_i = 7480000
values_log_epsilon = list(np.arange(-10*k_i, -k_i/10, k_i/100))
best_k = 1
for k in values_log_epsilon:
    y_pred = classify(X_test, distribution, log_epsilon=k)
    if f1_score(y_test, y_pred) > max_score:
        max_score = f1_score(y_test, y_pred)
        best_k = k
    
# Calculate accuracy of the prediction.
accuracy = sum(np.asarray(y_test == y_pred))/np.asarray(y_test == y_pred).shape[0]
print("Accuracy:", accuracy)
print("log epsilon : ", best_k)
print("F1 Score : ", max_score)

Accuracy: 0.32369768142125865
log epsilon :  -6657200.0
F1 Score :  0.6710182767624021


**Discussions:**
- Can you please try to improve the performance of this approach?
    - For example, by normalizating the data?
    - By smoothing the data?
    - By reducing feature number?
    - etc.
- The parameter log_epsilon defines the threshold we use for making classification. What happens if you change it?
- Could you discuss how we should get the best value for this parameter?

To get the best value of the log_epsilon parameter, we can try to get an approximate value by hand at first to have an estimation of the range of the optimal value. Then we can iterate over the range to see which value gives the best results.

## Local outiler factor (LOF)

The local outlier factor (LOF) algorithm computes the local density deviation of a given data point with respect to its neighbors. It considers as outliers the samples that have a substantially lower density than their neighbors. You can easiliy implement LOF in scikit-learn ([tutorial](https://www.datatechnotes.com/2020/04/anomaly-detection-with-local-outlier-factor-in-python.html)).

Please implement local outlier factor (LOF) algorithm on the dataset of `20240325_155003`. You can try first to detect the failure of motor 1 using this model. Please calculate the accuracy score of your prediction.

In [8]:
from sklearn.neighbors import LocalOutlierFactor
sys.path.append('../../supporting_scripts/WP_1')
base_dictionary = '../../dataset/training_data/'
dictionary_name = '20240325_155003'
path = base_dictionary + dictionary_name
df_data = read_all_csvs_one_test(path, dictionary_name)

features_to_keep = ["data_motor_1_voltage", "data_motor_1_temperature", "data_motor_1_position"]

X = df_data.loc[:, features_to_keep]
y_test = df_data['data_motor_1_label']

best_score = 0
best_neighbors = 0
best_y = 0

for n in range(180, 230):

    lof = LocalOutlierFactor(n_neighbors=n) 
    y_pred = lof.fit_predict(X)


    for k in range(len(y_pred)):
        if y_pred[k] == -1:
            y_pred[k] = 1
        else:
            y_pred[k] = 0

    if f1_score(y_pred, y_test) > best_score:
        best_score = f1_score(y_pred, y_test)
        best_neighbors = n 
        best_y = y_pred

print("F1 score : ",best_score)
print("Number of neighbors ", best_neighbors)
print("Accuracy : ", sum(y_pred == y_test)/len(y_pred))

F1 score :  0.2932288752098489
Number of neighbors  206
Accuracy :  0.8071256764882742


# Task 2 Develop a cross validation pipeline to evaluate the performance of the model.

The idea of cross validation is to split the data into k subsets and use one of them as the test set and the rest as the training set. The performance of the model is evaluated only on the test dataset, while the model is trained on the training dataset. By doing this, we ensure that the evaluation of the model is independent from the training of the model. Therefore, we can detect if the model is overfitted.

## k-fold cross validation

Here, we use motor 1 as an example to develop a pipeline for cross validation. Below, you have a script that read the data, extract features and get the labels.

1. Use sk-learn to split the data into training and testing sets, using a k-fold cross validation with k=5. (Hint: This is a routine task which can be answered easily by language models like chatgpt. You can try prompt like this: `Generate a code in python to split the data X and y into training and testing sets, using a k-fold cross validation with k=5.`)
2. Then, train a basic logistic regression model, without hyper-parameter tuning on the training set, and use the testing set to evaluate the performance of the model (calculate accuracy, precision, recall, and F1 score). 
3. Finally, train a logistic regression model, but use the entire dataset X and y as training data. Then, use the trained model to predict the labels of the same dataset (X). Compare the results with the previous step, and discuss why we should use cross validation to evaluate the performance of the model.

In [11]:
import sys
sys.path.append('../../supporting_scripts/WP_1')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from utility import read_all_test_data_from_path
import pandas as pd

# Specify path to the dictionary.
# Define the path to the folder 'collected_data'
base_dictionary = '../../dataset/training_data/'
# Read all the data
df_data = read_all_test_data_from_path(base_dictionary, is_plot=False)

# Extract the features for motor 1: You should replace the features with the ones you have selected in WP1.
X = df_data[['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage']]

# Get the label
y = df_data['data_motor_1_label']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### With validation

clf = LogisticRegression()
clf.fit(x_train, y_train)
score_valid = clf.score(x_test, y_test)

### Without validation

clf = LogisticRegression()
clf.fit(x_train, y_train)
score_without_valid = clf.score(x_train, y_train)

print("Accuracy with testing on the validation set : ", score_valid)
print("Accuracy with testing on the training set : ", score_without_valid)

Accuracy with testing on the validation set :  0.9650216229966929
Accuracy with testing on the training set :  0.9654339046649919


Write your discussions here:

Most of the time, the accuracy is higher when we test our classifier on the training set, which is obvious because we train the model so that it can pe efficient on this set.  

# Task 3: Develop classification-based fault detection models

In this task, you are supposed to experiment different classification-based fault detection models to get best F1 score. Please use the 5-fold cross-validation to calculate the best F1 score. You are free to try different models, whether they are discussed in the class or not. To simply your work, you can use the models existed in [scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

Please report all the models you tried, how to you tune their hyperparameters, and the corresponding F1 score. Please note that if you would like to tune the hyperparameter, you can use the `GridSearchCv` function in scikit-learn, but you should use it only on the training dataset.

## SVM with hyperparameter tuning

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from utility import read_all_test_data_from_path
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import sys
sys.path.append('../../supporting_scripts/WP_1')

### Get all the data
base_dictionary = '../../dataset/training_data/'
df_data = read_all_test_data_from_path(base_dictionary, is_plot=False)

X = df_data[['data_motor_1_temperature', 'data_motor_2_temperature', 'data_motor_3_temperature', 'data_motor_6_temperature', 'data_motor_3_position']]
y = df_data['data_motor_1_label']


#Separate the data between train/validation sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Preprocessing steps 

# Scaling 
scaler = StandardScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train))
x_test = pd.DataFrame(scaler.transform(x_test))

# Smoothing the data
X_train_smooth = x_train.copy()
X_test_smooth = x_test.copy()
window_size = 10
for col in x_train.columns:
    X_train_smooth[col] = x_train[col].rolling(window=window_size).mean()
    X_test_smooth[col] = x_test[col].rolling(window=window_size).mean()
X_train_smooth = X_train_smooth.loc[window_size:, :]
X_test_smooth = X_test_smooth.loc[window_size:, :]

x_train = X_train_smooth.copy()
x_test = X_test_smooth.copy()
y_train = y_train.iloc[window_size:]
y_test = y_test.iloc[window_size:]

### Classifier

clf = SVC(kernel="rbf", class_weight="balanced") # We change the weight because our dataset is unbalanced
grid = {"C" : np.linspace(10, 50, 1)} #We want high bvalue of C so our model classify correctly the rows where y = 1

clf_cv = GridSearchCV(clf, grid, scoring=f1_score, n_jobs=-1)
clf_cv.fit(x_train, y_train)
y_pred = clf_cv.predict(x_test)

print(clf_cv.best_params_)
print(classification_report(y_test, y_pred))


c:\Users\maxde\lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


{'C': 10.0}
Ratio de 1 théorique :  0.035277636271013754
Ratio de 1 pour les prédictions :  0.3706062149770759
              precision    recall  f1-score   support

           0       0.98      0.64      0.77      7575
           1       0.06      0.62      0.11       277

    accuracy                           0.64      7852
   macro avg       0.52      0.63      0.44      7852
weighted avg       0.95      0.64      0.75      7852



XGBoost

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA 
import random as rd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from utility import read_all_test_data_from_path
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

import sys
sys.path.append('../../supporting_scripts/WP_1')

### Get all the data
base_dictionary = '../../dataset/training_data/'
df_data = read_all_test_data_from_path(base_dictionary, is_plot=False)

features_not_to_keep = ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label', 'data_motor_5_label', 'data_motor_6_label', "time", "test_condition"]

X = df_data[[col for col in df_data.columns if col not in features_not_to_keep]]
y = df_data['data_motor_1_label']


#Separate the data between train/validation sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Preprocessing steps 

# Scaling 
scaler = StandardScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), index=y_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), index=y_test.index)

# Smoothing the data
X_train_smooth = x_train.copy()
X_test_smooth = x_test.copy()
window_size = 10
for col in x_train.columns:
    X_train_smooth[col] = x_train[col].rolling(window=window_size).mean()
    X_test_smooth[col] = x_test[col].rolling(window=window_size).mean()
X_train_smooth = X_train_smooth.iloc[window_size:, :]
X_test_smooth = X_test_smooth.iloc[window_size:, :]

x_train = X_train_smooth.copy()
x_test = X_test_smooth.copy()

y_train = y_train.iloc[window_size:]
y_test = y_test.iloc[window_size:]

# Here we delete some rows where y = 1 so that the dataset is more balanced
n, m = x_train.shape 
indice_y_1 = list(y_train == 1)
proportion_a_garder = 0.1 # This is the share of rows whe are keeping

indice_a_garder = indice_y_1
for k in range(len(indice_a_garder)):
    if indice_a_garder[k] == False:
        a = rd.random()
        if a < proportion_a_garder:
            indice_a_garder[k] = True
        
x_train = x_train[indice_a_garder]
y_train = y_train[indice_a_garder]

# Boosting

xgb = GradientBoostingClassifier()

xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      7596
           1       0.12      0.16      0.14       256

    accuracy                           0.93      7852
   macro avg       0.55      0.56      0.55      7852
weighted avg       0.94      0.93      0.94      7852



Random Forest with hyperparameter tuning

In [24]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from utility import read_all_test_data_from_path
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

import sys
sys.path.append('../../supporting_scripts/WP_1')

### Get all the data
base_dictionary = '../../dataset/training_data/'
df_data = read_all_test_data_from_path(base_dictionary, is_plot=False)

features_not_to_keep = ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label', 'data_motor_5_label', 'data_motor_6_label', "time", "test_condition"]

X = df_data[[col for col in df_data.columns if col not in features_not_to_keep]]
y = df_data['data_motor_1_label']


#Separate the data between train/validation sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Preprocessing steps 

# Scaling 
scaler = StandardScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), index=y_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), index=y_test.index)

# Smoothing the data
X_train_smooth = x_train.copy()
X_test_smooth = x_test.copy()
window_size = 10
for col in x_train.columns:
    X_train_smooth[col] = x_train[col].rolling(window=window_size).mean()
    X_test_smooth[col] = x_test[col].rolling(window=window_size).mean()
X_train_smooth = X_train_smooth.iloc[window_size:, :]
X_test_smooth = X_test_smooth.iloc[window_size:, :]

x_train = X_train_smooth.copy()
x_test = X_test_smooth.copy()

y_train = y_train.iloc[window_size:]
y_test = y_test.iloc[window_size:]

# Random Forest with hyperparameter tuning

# Dictionnary for the weights (unbalanced datasets)
dico = {0 : 10**(-8.5), 1 : 1}

clf = RandomForestClassifier(class_weight=dico)
grid = {"n_estimators" : [10, 50, 100]}

clf_cv = GridSearchCV(clf, grid)
clf_cv.fit(x_train, y_train)
y_pred = clf_cv.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.11      0.20      7560
           1       0.04      0.96      0.08       292

    accuracy                           0.14      7852
   macro avg       0.51      0.53      0.14      7852
weighted avg       0.95      0.14      0.20      7852



## Summary of the results

Please add a table in the end, summarying the results from all the models (including the unsupervised learning models). Please write a few texts to explain what is the best model you got, its performance, and how could you further improve it.

Please note our classifiers randomly select data each time they are run, thus the following results might differ from a few percentage.

| Model   | Accuracy | Precision | Recall | F1   |
|---------|----------|-----------|--------|------|
| SVM     |   64%    |   6%      |  62%   | 11%  |
| XGB     |   93%    |   12%     |  16%   | 14%  |
|Rd Forest|   14%    |   4%      |  96%   | 8%   |
